# SI670 Project: Boston Health Inspections
## Part 1: Data Cleaning and Feature Extraction

### Action Plan
- Get the relevant data (permits and inspections data for Boston)
    - https://data.boston.gov/dataset/food-establishment-inspections
    - https://data.boston.gov/dataset/approved-building-permits
- Merge the datasets based on PK --> property_id
- Data Cleaning: removing redundant/irrelevant features, adapt data types for our classification task. Removing temporal data (out of scope), 
- Feature Extraction: encoding, scaling, labeling features to expand dimensionality (categorical data encoding, numerical data scaling)      

- GOAL: build a binary classifier that determines whether a given inspection passed/failed.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler, LabelEncoder

import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
## Reading in inspections data df (626105, 26)
inspections = pd.read_csv("data/inspections.csv")
inspections.shape

/opt/anaconda3/envs/670proj/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (1,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(626105, 26)

In [ ]:
## Reading inpermits data df (481196, 22)
permits = pd.read_csv("data/building_permits.csv")
permits.shape

(481196, 22)

### Current Inspections Features

In [ ]:
inspections.columns

Index(['businessname', 'dbaname', 'legalowner', 'namelast', 'namefirst',
       'licenseno', 'issdttm', 'expdttm', 'licstatus', 'licensecat',
       'descript', 'result', 'resultdttm', 'violation', 'viollevel',
       'violdesc', 'violdttm', 'violstatus', 'statusdate', 'comments',
       'address', 'city', 'state', 'zip', 'property_id', 'location'],
      dtype='object')

#### Feature Extraction Ideas for Inspections Data
- Drop: business name, dbaname, legal owner, namelast, namefirst, issdttm, expdttm, resultdttm, comments, violation codes, violation level, violation description, violation status, address, city, state, latitude/longitude
- Encode: licensecat, result (binary target variable), zip code, 
- Primary key: property ID.

In [ ]:
inspections.dtypes

businessname     object
dbaname          object
legalowner       object
namelast         object
namefirst        object
licenseno         int64
issdttm          object
expdttm          object
licstatus        object
licensecat       object
descript         object
result           object
resultdttm       object
violation        object
viollevel        object
violdesc         object
violdttm         object
violstatus       object
statusdate       object
comments         object
address          object
city             object
state            object
zip              object
property_id     float64
location         object
dtype: object

#### Cleaning Up Inspection Data

In [ ]:
## also dropping business name since it could be PII
cols_to_drop = ["dbaname", "legalowner", "namelast", "namefirst", "issdttm", "expdttm", "resultdttm", 
                "comments", "address", "location", "businessname"]

clean_inspections = inspections.drop(cols_to_drop, axis=1)

In [ ]:
## with initial dropped columns, we have 15 total columns
clean_inspections.shape

(626105, 15)

In [ ]:
## there are some where property_id is NaN, so we gotta drop these (reduce the data) -- SHAPE INDICATED
clean_inspections = clean_inspections[~clean_inspections.property_id.isna()]

## also gotta change property_id to int type
clean_inspections.property_id = clean_inspections.property_id.astype("int32")

clean_inspections.shape

(491335, 15)

In [ ]:
## filtering only ones that PASSED or FAILED an inspection (for binary classifier)
clean_inspections = clean_inspections[(clean_inspections.result == "HE_Pass") | (clean_inspections.result == "HE_Fail")]
clean_inspections.shape

(364904, 15)

In [ ]:
clean_inspections.sample(2)

,licenseno,licstatus,licensecat,descript,result,violation,viollevel,violdesc,violdttm,violstatus,statusdate,city,state,zip,property_id
603172,22543,Inactive,RF,Retail Food,HE_Pass,08-3-305-307.11,*,Food Protection,2010-04-07 10:53:54,Pass,2010-04-08 08:24:20,Dorchester,MA,02121,17643
407533,117274,Active,FT,Eating & Drinking w/ Take Out,HE_Pass,37-6-201.11,*,Walls/Ceilings Designed Constructed Installed,2017-09-05 11:33:50,Pass,2017-09-06 08:40:33,Boston,MA,2108,170408


### Current Building Permits Features

In [ ]:
permits.columns

Index(['permitnumber', 'worktype', 'permittypedescr', 'description',
       'comments', 'applicant', 'declared_valuation', 'total_fees',
       'issued_date', 'expiration_date', 'status', 'owner', 'occupancytype',
       'sq_feet', 'address', 'city', 'state', 'zip', 'property_id',
       'parcel_id', 'lat', 'long'],
      dtype='object')

#### Feature Extraction Ideas for Permits Data
- Drop: worktype, permittypedescr, description, comments, applicant, occupancy type (all food establishments), owner, address, city, state, zip (redundant), parcel_id, lat/long
- Encode: permit status, 
- Scale: square footage, total fees, declared valuation
- property_id is the primary key (to merge)

In [ ]:
permits.dtypes

permitnumber           object
worktype               object
permittypedescr        object
description            object
comments               object
applicant              object
declared_valuation    float64
total_fees            float64
issued_date            object
expiration_date        object
status                 object
owner                  object
occupancytype          object
sq_feet               float64
address                object
city                   object
state                  object
zip                    object
property_id            object
parcel_id              object
lat                   float64
long                  float64
dtype: object

#### Cleaning Up Permits Data

In [ ]:
## cols to drop in permits data, at least initially
drop_these = ["worktype", "permittypedescr", "description", "comments", "applicant", "owner", "address", "city", "state", "zip", "parcel_id", "lat", "long"] 
clean_permits = permits.drop(drop_these, axis=1)

In [ ]:
clean_permits.shape

(481196, 9)

In [ ]:
## again, property_id's could be NaN, so reduce these by filtering
clean_permits = clean_permits[~clean_permits.property_id.isna()]
clean_permits = clean_permits[clean_permits.property_id != " "]

## gotta change dtype of property id from object to int --> GONNA HAVE TO DO A BUNCH OF DTYPE CONVERSIONS LATER
clean_permits.property_id = clean_permits.property_id.astype("int32")

clean_permits.shape

(472248, 9)

In [ ]:
clean_permits

,permitnumber,declared_valuation,total_fees,issued_date,expiration_date,status,occupancytype,sq_feet,property_id
0,A100071,40000.00,429.0,2011-11-04 11:04:58,2012-05-04 00:00:00,Open,Comm,170.0,17268
1,A1001012,5000.00,70.0,2020-06-01 14:08:47,2020-12-01 00:00:00,Open,1-3FAM,0.0,113443
2,A1001201,74295.75,803.0,2019-11-13 13:38:56,2020-05-13 00:00:00,Open,Multi,0.0,129994
3,A100137,15000.00,206.0,2013-01-03 14:13:09,2013-07-03 00:00:00,Open,1-2FAM,0.0,149852
4,A1001913,1.00,33.0,2019-10-18 09:21:00,2020-04-18 00:00:00,Open,Comm,0.0,20073
...,...,...,...,...,...,...,...,...,...
481191,U49993435,0.00,50.0,2019-09-25 11:36:11,2020-03-25 00:00:00,Open,Mixed,0.0,154961
481192,U49993435,0.00,50.0,2019-09-25 11:36:11,2020-03-25 00:00:00,Open,Mixed,0.0,154961
481193,U49993492,1000.00,53.0,2019-09-12 13:07:00,2020-03-12 00:00:00,Open,Comm,0.0,424193
481194,U4999352,2700.00,53.0,2012-05-30 14:28:59,2012-11-30 00:00:00,Open,1-2FAM,0.0,24047


### Merging Our Data
- Merging our data based on the property_id column
- pandas .merge inner, since property_id repeats we get many more rows than the original datasets
- 


In [ ]:
## inner join on property_id
merged_data = clean_inspections.merge(clean_permits, how="inner", left_on="property_id", right_on="property_id")

In [ ]:
merged_data.shape

(45884114, 23)

In [ ]:
merged_data.sample(2)#.sort_values("issued_date")

,licenseno,licstatus,licensecat,descript,result,violation,viollevel,violdesc,violdttm,violstatus,...,zip,property_id,permitnumber,declared_valuation,total_fees,issued_date,expiration_date,status,occupancytype,sq_feet
35180107,163952,Active,FT,Eating & Drinking w/ Take Out,HE_Fail,29-5-201/02.11,*,Installed and Maintained,2017-02-01 11:56:40,Fail,...,02199,157244,EFA588042,400.0,30.0,2016-05-06 13:14:34,2016-11-06 00:00:00,Open,Comm,0.0
24898849,129193,Inactive,MFW,Mobile Food Walk On,HE_Pass,NaN,NaN,NaN,,NaN,...,02128,0,SF29911,1000.0,17.0,2010-06-16 13:04:26,2010-12-16 00:00:00,Open,1-2FAM,48.0


### Merged Data: Further Pre-processing
- Refining our features, performing scaling on appropriate columns
- Based on previous goals of dropping/encoding/labeling/scaling, we perform all of our pre-processing on the data
- Due to the volume, our process was done using only a small subset of data. We later used the same process on larger samples. 

In [ ]:
merged_data.columns

Index(['licenseno', 'licstatus', 'licensecat', 'descript', 'result',
       'violation', 'viollevel', 'violdesc', 'violdttm', 'violstatus',
       'statusdate', 'city', 'state', 'zip', 'property_id', 'permitnumber',
       'declared_valuation', 'total_fees', 'issued_date', 'expiration_date',
       'status', 'occupancytype', 'sq_feet'],
      dtype='object')

In [ ]:
## get a sample of the data to speed up the process
sample_data = merged_data.sample(5000)

In [ ]:
sample_data.head(2)

,licenseno,licstatus,licensecat,descript,result,violation,viollevel,violdesc,violdttm,violstatus,...,zip,property_id,permitnumber,declared_valuation,total_fees,issued_date,expiration_date,status,occupancytype,sq_feet
22514804,25669,Inactive,FT,Eating & Drinking w/ Take Out,HE_Pass,03-3-501.14,***,Cooling,2008-10-28 15:39:29,Pass,...,02127,0,E315886,460000.0,1895.0,2013-12-30 10:23:06,2014-06-30 00:00:00,Closed,Comm,130000.0
14474045,68113,Active,MFW,Mobile Food Walk On,HE_Pass,NaN,NaN,NaN,,NaN,...,2128,0,SF38231,2950.0,50.0,2010-08-13 12:45:11,2011-02-13 00:00:00,Open,Other,3600.0


In [ ]:
### WE NEED TO DO THIS FOR THE WHOLE DATAFRAME LATER, USING A SAMPLE SUBSET FOR NOW

# ## drop ones without zip, also need to make them all integers
# sample_data = sample_data.drop(sample_data[sample_data.zip == " "].index)
# sample_data["zip"] = sample_data.zip.apply(lambda x: int(x))

# ## drop ones without occupancy type " "
# sample_data.occupancytype.unique()
# sample_data = sample_data.drop(sample_data[sample_data.occupancytype == " "].index)


In [ ]:
### with all merged data

## drop ones without zip, also need to make them all integers
merged_data.zip = merged_data.zip.fillna(" ")
merged_data = merged_data.drop(merged_data[merged_data.zip == " "].index)
merged_data["zip"] = merged_data.zip.apply(lambda x: int(x))

## drop ones without occupancy type " "
merged_data.occupancytype.unique()
merged_data = merged_data.drop(merged_data[merged_data.occupancytype == " "].index)

In [ ]:
merged_data.columns

Index(['licenseno', 'licstatus', 'licensecat', 'descript', 'result',
       'violation', 'viollevel', 'violdesc', 'violdttm', 'violstatus',
       'statusdate', 'city', 'state', 'zip', 'property_id', 'permitnumber',
       'declared_valuation', 'total_fees', 'issued_date', 'expiration_date',
       'status', 'occupancytype', 'sq_feet'],
      dtype='object')

In [ ]:
merged_data.sample(2)

,licenseno,licstatus,licensecat,descript,result,violation,viollevel,violdesc,violdttm,violstatus,...,zip,property_id,permitnumber,declared_valuation,total_fees,issued_date,expiration_date,status,occupancytype,sq_feet
18694748,76348,Active,MFW,Mobile Food Walk On,HE_Fail,NaN,NaN,NaN,,NaN,...,2128,0,E64722,400.0,30.0,2011-02-18 14:00:59,2011-08-18 00:00:00,Closed,Comm,0.0
27360052,75234,Inactive,MFW,Mobile Food Walk On,HE_Pass,M-2-103.11,***,PIC Performing Duties,2017-09-29 11:35:54,Pass,...,2128,0,E54147,19000.0,90.0,2010-12-03 08:35:16,2011-06-03 00:00:00,Closed,1-3FAM,0.0


In [ ]:
def cleanup_merged_data(merged_data):
    """
    Function cleans our merged data (whether full or sampled) by dropping previously mentioned columns, one-hot encoding categorical variables,
    dropping the encoded variables, and scaling relevant continuous features. 
    :input: merged_data sample/full
    :output: cleaned dataframe for model building
    """
    ## we need to drop: viollevel, violation, violdttm, status_date, violdesc, city, 
    ## permit_number, issued_date, expiration_date, status
    to_drop = ["viollevel", "violation", "violdttm", "violdesc", "statusdate", "permitnumber", 
               "issued_date", "expiration_date", "status"]
    
    ## drop the columns we no longer need
    filt_df = merged_data.drop(to_drop, axis=1)
    
    ## we need to encode: descript, licensecat, zip, occupancy_type (pandas dummies)
    licensestatus_dummies = pd.get_dummies(filt_df.licstatus)
    description_dummies = pd.get_dummies(filt_df.descript)
    licensecat_dummies = pd.get_dummies(filt_df.licensecat)
    zip_dummies = pd.get_dummies(filt_df.zip)
    occ_dummies = pd.get_dummies(filt_df.occupancytype)

    ## join the dummies based on index, then drop the columns
    filt_df = filt_df.join(licensestatus_dummies)
    filt_df = filt_df.join(description_dummies)
    filt_df = filt_df.join(licensecat_dummies)
    filt_df = filt_df.join(zip_dummies)
    filt_df = filt_df.join(occ_dummies)
    
    # dropped the encoded columns
    filt_df.drop(["descript", "licensecat", "zip", "occupancytype"], axis=1, inplace=True)
    
    
    ## we need to scale: declared_valuation, total_fees, square feet
    valuation_scaled = RobustScaler().fit_transform(filt_df.declared_valuation.values.reshape(-1,1))
    fees_scaled = RobustScaler().fit_transform(filt_df.total_fees.values.reshape(-1,1))
    sqft_scaled = RobustScaler().fit_transform(filt_df.sq_feet.values.reshape(-1,1))
    
    filt_df["declared_valuation"] = valuation_scaled
    filt_df["total_fees"] = fees_scaled
#     filt_df["sq_feet"] - sqft_scaled ## doesn't work for some reason, scaling done in model building stage

    ## need to label encode licstatus, result
    licstatus_labels = LabelEncoder().fit_transform(filt_df.licstatus) ##1 inactive, 0 active
    filt_df["licstatus"] = licstatus_labels
    
    result_labels = LabelEncoder().fit_transform(filt_df.result) ##1 pass, 0 fail
    filt_df["result"] = result_labels
    
    ## now making final removals of features, making decisions as to what to use
    # drop: violstatus (already have result), state, city (since we already have zip)
    filt_df.drop(["violstatus", "state", "city"], axis=1, inplace=True)
    
    return filt_df
    


In [ ]:
## getting a preliminary 'clean_df' 
prelim_clean_df = cleanup_merged_data(merged_data)

In [ ]:
## licstatus: 1 inactive, 0 active
## result: 1 pass, 0 fail
prelim_clean_df


,licenseno,licstatus,result,property_id,declared_valuation,total_fees,sq_feet,Active,Deleted,Inactive,...,5unit,6unit,7More,7unit,COMM,Comm,Mixed,Multi,Other,VacLd
0,87059,0,0,77476,-0.212766,-0.005747,0.0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,87059,0,0,77476,-0.153191,-0.195402,0.0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,87059,0,0,77476,-0.106383,-0.218391,0.0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
5,87059,0,0,77476,-0.042553,0.235632,0.0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
6,87059,0,0,77476,-0.106383,-0.057471,0.0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45884109,27260,0,1,88457,0.553191,0.000000,700.0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
45884110,27260,0,1,88457,1.914894,0.574713,0.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
45884111,27260,0,1,88457,-0.148936,-0.172414,900.0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
45884112,27260,0,1,88457,-0.208511,-0.201149,0.0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


### Write our full merged df as a CSV file for efficiency purposes

In [ ]:
# prelim_clean_df.to_csv("cleaned_data.gz", compression="gzip", chunksize=10000)

In [ ]:
# prelim_clean_df.to_csv("cleaned_data.csv")